# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [7]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [8]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [9]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1
def scroll_down(self):
    last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    while True:
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load the page.
        time.sleep(2)
        # Calculate new scroll height and compare with last scroll height.
        new_height = self.driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
                break
        last_height = new_height

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [11]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [12]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 晶悅將公布「重大訊息」25日起暫停交易　傳桃園老飯店恐歇業！

中央社訊
上櫃公司晶悅今天公告因有重大事項待公布，25日暫停交易，由於旗下桃園老字號飯店「晶悅國際飯店」先前傳出客房部4月歇業，市場猜測，明天公布的重大訊息恐與此有關。
櫃買中心今天指出，晶悅公司上櫃有價證券自3月25日起暫停交易。
晶悅國際飯店因有重大事項待公布，櫃買中心公告該公司上櫃有價證券自3月25日起暫停在證券商營業處所買賣，公司將於前開重大訊息公布後，再申請恢復交易。
消息指出，晶悅將自4月1日起飯店停止訂房，員工予以資遣。桃園勞動局就服處日前也接獲通報，晶悅國際飯店將資遣約36人。
晶悅先前澄清表示，關於晶悅國際飯店是否歇業須等董事會決議後確認，預計於董事會決議後公告。
疫情重創觀光業，台中亞緻大飯店扛不住壓力宣布正式熄燈，多家飯店也傳出休無薪假、裁員或是暫停歇業。
桃園晶悅國際飯店除了傳出客房部將歇業外，也受到三發地產關係企業近來「家庭宮鬥」的風暴波及。
晶悅國際飯店前身為桃園大飯店，有40年歷史，是桃園地區的老字號飯店，設置388間客房，2013年由三發地產董事長鍾俊榮透過三發地產旗下的投資公司收購後，改名為晶悅國際飯店，並由董娘楊顯玲擔任晶悅董事長。
根據晶悅財報顯示，2014、2015年每股盈餘（EPS）都有1元以上的水準，不過2016年起開始走下坡，2018年出現首度虧損，每股虧0.47元，去年前3季每股也虧損0.09元。
----------------------------------------------------------------------

[1] 國產署標租國有地供太陽能光電使用　商機破4億元

▲太陽能光電。（圖／記者湯興漢攝）
記者吳靜君／台北報導
國產署今（24）日標租今年第一批國有非公用土地供太陽能光電設備使用，有三案順利標脫，預估20年商機可達4.36億元。
原本預計標租四塊土地，包含台中、雲林、高雄以及嘉義，但是因為雲林的地因為遭非法占用，所以宣告停標，等問題處理完畢，再決定是否列標。而其他三塊土地則是順利標租，且標封起碼兩封以上，甚至最高可達6封，顯見國有地標租光電設備使用愈來愈熱絡。
順利標

▲台北國際電腦展（COMPUTEX TAIPEI）為每年重頭戲之一。（圖／資料照）
記者余弦妙／台北報導
因應新冠肺炎(COVID-19)疫情影響，外貿協會與台北市電腦公會決定，原訂今年6月2日至6日舉辦的COMPUTEX，將延期至9月辦理。
貿協指出，由於新冠肺炎目前已累積超過30萬人確診、分布於全球167國，已嚴重衝擊經濟、貿易、交通運輸與會展觀光等各行各業。因此，COMPUTEX主辦單位外貿協會秘書長葉明水及台北市電腦公會總幹事杜全昌共同宣布，基於維護參展廠商及參觀者的健康與安全，顧及參展效益與維繫COMPUTEX品牌形象，今年展期確定延期。
貿協表示，COMPUTEX台北國際電腦展今年邁入40周年，該展是全球前三大科技展，與美國CES、德國CeBIT並列，也是全球科技業的新技術、新品發表大會，全球的科技業者均會來台觀摩、採購，是全球最重要的電子採購展，每年來台採購的國際買主逾4萬人，每年也均對台灣的飯店業、觀光業帶來極大商機。
但由於今年受到疫情影響，為了持續服務全球有需求的參展廠商及買主，並顧及所有與會者之健康安全，今年COMPUTEX將延期至9月28日至30日，於台北南港展覽館2館舉行，將呈現5G與通訊、智慧解決方案、電競及創新與新創（InnoVEX）等重點主題。
貿協也說，依據麥肯錫最新疫情報告，中國與東亞的疫情在第2季初期會受到控制，歐美地區疫情也會在6月趨緩，外貿協會將透過全球63個駐外單位加強洽邀全球買主來台參觀9月的COMPUTEX。
此外，主辦單位將在6月原檔期期間，在台灣經貿網辦理COMPUTEX線上展覽，廣邀全球買主參加視訊採購洽談會，及邀請國內外指標廠商進行線上新產品發表會，同期將邀請國內外資通訊指標性業者，以Webinar型式分享5G與AIoT等智慧科技應用與最新趨勢。
----------------------------------------------------------------------

[11] 紓困奏效！歐股早盤強勢上揚逾4%、德國指數領漲566點或6%

▲歐股今(24)日早盤受亞股鼓舞，齊呈現漲幅。（圖／路透社）
記者謝仁傑／綜合報導
歐洲股市今（24）日早盤受亞股鼓舞，開高後紛紛呈現上漲的態勢，主要股市皆漲逾4％；英股、法股、瑞士股開盤直線急升，德股漲最多達6％，西班牙股市則維持漲4％上下。
隨著德國